<h1 align='center'> 영상처리 프로그래밍 실습 8</h1>

<h6 align='right'> 2022. 5. 10. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름:   &nbsp;&nbsp;          학번:    &nbsp;&nbsp;         학과(전공):
    
</div>

- JupyterLab 문서의 최신 버전은 [JupyterLab Documentation](https://jupyterlab.readthedocs.io/en/stable/index.html#/)을  참고하라

- Markdown은 [Markdown Guide](https://www.markdownguide.org/)를 참고하라.
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)

* 제출 마감: 5월 16일 (월) 오후 6:00까지 최종본을 SmartLEAD제출


In [ ]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

## 지난 주 문제 1. 

예제 2의 프로그램을 다음과 같이 변경하라.

- 이동 평균 필터링을 가우시안 필터링으로 변경하라.
- 가우시안 커널의 표준편차인 sigmaX와 sigmaY를 개별적으로 조정할 수 있는 trackBar interface를 2 개 제공한다. 단, 표준 편차의 최댓값은 10.0이 되도록 한다.


In [ ]:
def Gaussian_filtering(file_name):
    def onGaussianFiltering(val):
        nonlocal img2, ksize, sigmaX, sigmaY
        ksize = val // 2 * 2 + 1
        img_filtered = cv2.GaussianBlur(img, (ksize, ksize), sigmaX=sigmaX, sigmaY=sigmaY)
        img2 = np.hstack((img, img_filtered))
        cv2.imshow(winName, img2)
            
    def onSigmaX(val):
        nonlocal sigmaX, sigmaY
        sigmaX = val / sigma_scale_factor
        v = ksize
        onGaussianFiltering(v)

    def onSigmaY(val):
        nonlocal sigmaX, sigmaY
        sigmaY = val / sigma_scale_factor
        v = ksize
        onGaussianFiltering(v)

    
    from os.path import exists
    ksize = 3
    sigmaX = 0.1
    sigmaY = 0.1
    sigma_scale_factor = 10
    if exists(file_name):
        img = cv2.imread(file_name)
    else:
        print(f"Image file '{file_name}' does not exist.")
        return
    img2 = np.hstack((img, img))

    winName = "Gaussian Filtering"
    cv2.namedWindow(winName, flags=1|cv2.WINDOW_GUI_NORMAL )
    cv2.imshow(winName, img2)

    cv2.createTrackbar('kernel size', winName, 1, 51, onGaussianFiltering)
    cv2.createTrackbar('sigma_x', winName, int(sigmaX*sigma_scale_factor), 101, onSigmaX)
    cv2.createTrackbar('sigma_y', winName, int(sigmaY*sigma_scale_factor), 101, onSigmaY)
    cv2.setTrackbarMin('kernel size', winName, 1)
    cv2.setTrackbarMin('sigma_x', winName, 1)
    cv2.setTrackbarMin('sigma_y', winName, 1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    

In [ ]:
Gaussian_filtering('grass.jpg')

## 예제 1.

다음 프로그램을 실행하라.


In [ ]:
from scipy import misc
ascent = misc.ascent()
plt.imshow(ascent, cmap='gray')
plt.show()

In [ ]:
ascent.dtype, ascent.min(), ascent.max()

In [ ]:
img = img.astype(np.uint8)

### 예제 2. 
예제 1의 영상에 아래 두 커널을 적용해서 gradient의 크기를 구하고, 
cv2.imshow 함수를 이용해서 원 영상과 gradient 크기 영상을 나란히 표시하라.

In [ ]:
k_diff_x = np.array([[0, 0, 0], 
                [0, -1, 0], 
                [0, 1, 0]])
k_diff_y = np.array([[0, 0, 0], 
                [0, -1, 1], 
                [0, 0, 0]])

### 예제 3. 
예제 1의 영상에 두 개의 대각 방향의 Roberts 두 커널을 적용한 후에 에지의 크기를 구하고, 
cv2.imshow 함수를 이용해서 원 영상과 에지의 크기 영상을 나란히 표시하라.

### 예제 4. 
예제 2와 예제 3의 프로그램을 하나로 통합하여 edge_detection(img) 함수로 만들려고 한다.
단, trackBar interface를 이용해서 수평/수직 방향의 에지의 크기를 선택한다.
- 0: 원 영상 - 원 영상
- 1: 원 영상 - 수평/수직 방향의 에지 크기
- 2: 원 영상 - 대각 방향의  에지 크기


In [ ]:
def edge_detection(img):
    


In [ ]:
edge_detection(ascent)

### 예제 5. 
예제 4의 프로그램에 다음 기능을 추가하라.
- 0: 원 영상 - 원 영상
- 1: 원 영상 - 수평/수직 방향의 에지 크기
- 2: 원 영상 - 대각 방향의  에지 크기
- 3: 원 영상 - Prewitt 커널 에지 크기
- 4: 원 영상 - 4 방향 라플라시안 결과의 절대값
- 5: 원 영상 - 8 방향 라플라시안 결과의 절대값


In [ ]:
from os.path import exists
def edge_detection(img):
    


In [ ]:
edge_detection(ascent)

### 예제 6. 
예제 5의 프로그램을 원 영상에 가우시안 잡음을 추가한 후에 에지를 검출하는 프로그램으로 수정하라.

단, 왼쪽에는 가우시안 잡음이 포함된 영상을 표시하고, 오른쪽에는 예제 5에서 구한 에지 영상을 표시한다. 
또한, trackBar 인터페이스를 추가해서 가우시안 잡음의 분산을 조정할 수 있어야 한다.


In [ ]:
def edge_detection(img):
    


In [ ]:
edge_detection(ascent)

### 예제 7. Canny Edge detection

다음 도움말과 프로그램을 통해서 cv2.Canny() 함수의 사용법을 익혀보자.

In [ ]:
help(cv2.Canny)

In [ ]:
th1 = 80
th2 = 200
n = np.random.normal(0, 0.2, img.shape)
noisy_img_f = img/255 + n
noisy_img = (255*noisy_img_f).round(0).clip(0, 255).astype(np.uint8)
#img_filtered = cv2.Canny(noisy_img, th1, th2)
img_filtered = cv2.Canny(img, th1, th2)

plt.subplot(121)
plt.imshow(noisy_img, cmap='gray')
plt.subplot(122)
plt.imshow(img_filtered, cmap='gray')

### 문제 1. 다음 조건을 만족하는 함수 canny_edge_detection(filename) 함수를 작성하라.
- parameter로 주어진 영상 파일이 없으면 오류 메시지를 콘솔에 출력하고 종료한다.
- parameter로 주어진 영상 파일이 있으면 영상 파일을 읽고 원 영상과 cv2.Canny() 함수 실행 결과 영상을 나란히 표시한다.
- OpenCV의 trackbar 인터페이스를 통해서 cv2.Canny에 필요한 두 개의 thresholds를 조절한다.